# Volvemos al perceptron 2D

In [1]:
import numpy as np

In [2]:
# Repositorio de los datos
SEABORN_DATASETS_URL = 'https://raw.githubusercontent.com/mwaskom/seaborn-data/master'

datasource = np.DataSource()

In [3]:
# Cargar los datos
with datasource.open(SEABORN_DATASETS_URL + '/penguins.csv') as f:
    datos_pinguinos = np.genfromtxt(
    f, 
    delimiter=',', 
    names=True,
    dtype=[
           ('species', '|S10'), 
           ('island', '|S15'),
           ('bill_length_mm', float),
           ('bill_depth_mm', float),
           ('flipper_length_mm', float),
           ('body_mass_g', float),
           ('sex', '|S10')
    ])

In [4]:
datos_pinguinos

array([(b'Adelie', b'Torgersen', 39.1, 18.7, 181., 3750., b'MALE'),
       (b'Adelie', b'Torgersen', 39.5, 17.4, 186., 3800., b'FEMALE'),
       (b'Adelie', b'Torgersen', 40.3, 18. , 195., 3250., b'FEMALE'),
       (b'Adelie', b'Torgersen',  nan,  nan,  nan,   nan, b''),
       (b'Adelie', b'Torgersen', 36.7, 19.3, 193., 3450., b'FEMALE'),
       (b'Adelie', b'Torgersen', 39.3, 20.6, 190., 3650., b'MALE'),
       (b'Adelie', b'Torgersen', 38.9, 17.8, 181., 3625., b'FEMALE'),
       (b'Adelie', b'Torgersen', 39.2, 19.6, 195., 4675., b'MALE'),
       (b'Adelie', b'Torgersen', 34.1, 18.1, 193., 3475., b''),
       (b'Adelie', b'Torgersen', 42. , 20.2, 190., 4250., b''),
       (b'Adelie', b'Torgersen', 37.8, 17.1, 186., 3300., b''),
       (b'Adelie', b'Torgersen', 37.8, 17.3, 180., 3700., b''),
       (b'Adelie', b'Torgersen', 41.1, 17.6, 182., 3200., b'FEMALE'),
       (b'Adelie', b'Torgersen', 38.6, 21.2, 191., 3800., b'MALE'),
       (b'Adelie', b'Torgersen', 34.6, 21.1, 198., 4400., 

In [5]:
focus_species = [b'Adelie', b'Gentoo']
mask = np.isin(datos_pinguinos['species'], focus_species)

In [6]:
mask

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [7]:
data = np.vstack([
    datos_pinguinos[mask]['bill_length_mm'],
    datos_pinguinos[mask]['bill_depth_mm']
])

In [8]:
print(data.shape)
print(data[:, :10])

(2, 276)
[[39.1 39.5 40.3  nan 36.7 39.3 38.9 39.2 34.1 42. ]
 [18.7 17.4 18.   nan 19.3 20.6 17.8 19.6 18.1 20.2]]


In [9]:
labels = np.where(datos_pinguinos[mask]['species'] == b'Adelie', 1, -1)

In [12]:
print(labels.shape)
print(labels[:10])

(276,)
[1 1 1 1 1 1 1 1 1 1]


In [13]:
nan_mask = ~np.isnan(data).any(axis=0)

In [14]:
print(nan_mask)

[ True  True  True False  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  T

In [15]:
data = data[:, nan_mask]
labels = labels[nan_mask]

In [16]:
rng = np.random.default_rng(12345) # generador de numeros aleatorios

In [19]:
def train_perceptron(data, labels, max_iter=1000):
    ''' ejecuta el algoritmo del perceptron para los datos y sus etiquetas

    Parameters
    ----------
    data: List[List[Number]]
        datos de entrenamiento, lista de vetcores de dimensión 2
    labels: List[{-1, 1}]
        etiqueta de los datos, la entrada i corresponde a la etiqueta del dato i
    max_iter: int
        número máximo de iteraciones del algoritmo

    Returns
    -------
    Tuple(List[Number], Number)
        tupla con los valores para los pesos y el sesgo después del 
        entrenamiento

    Raises
    ------
    ValueError
        si el valor de max_iter no es entero positivo
    '''
    if not isinstance(max_iter, int) or max_iter <= 0:
        raise ValueError('max_iter debe ser entero positivo', max_iter)

    # Inicialiazacion
    gamma = rng.random()
    weights = rng.random(2)
    iter_ctn = 0
    pred_labels = np.where(weights@data+gamma >= 0, 1, -1) 
    missclassified = labels != pred_labels
    if np.all(~missclassified):
        return (weights, gamma)
    
    # entrenamiento
    while np.any(missclassified):
        idx = rng.choice(np.where(missclassified == True)[0], 1)
        l_star = labels[idx]
        data_star = data[:, idx].flatten()
        gamma = gamma + l_star
        weights = weights + l_star*data_star
        pred_labels = np.where(weights@data+gamma >= 0, 1, -1) 
        missclassified = labels != pred_labels

        iter_ctn = iter_ctn + 1
        if iter_ctn >= max_iter:
            break # esta palabra hace que salgas de cualquier ciclo
    
    return (weights, gamma)

In [20]:
w, g = train_perceptron(data, labels)

In [21]:
print('Pesos:', w)
print('Sesgo:', g)

Pesos: [-35.86718607  89.29830875]
Sesgo: [2.39110955]


In [22]:
pred_label = np.where(w@data+g >= 0, 1, -1) 

In [24]:
pred_label == labels

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,